In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
liewyousheng_minc2500_path = kagglehub.dataset_download('liewyousheng/minc2500')
# feyzazkefe_trashnet_path = kagglehub.dataset_download('feyzazkefe/trashnet')
# youssefhazemfarouk_sdscsc_path = kagglehub.dataset_download('youssefhazemfarouk/sdscsc')
# mazennafe3_trashnetttt_path = kagglehub.dataset_download('mazennafe3/trashnetttt')
# mazennafe3_trashdata_path = kagglehub.dataset_download('mazennafe3/trashdata')

print('Data source import complete.')

base_path = liewyousheng_minc2500_path

Using Colab cache for faster access to the 'minc2500' dataset.
Data source import complete.


In [3]:
# Define source directory and the new directory to hold selected classes
import shutil
import os
# source_dir = "/kaggle/input/minc2500/minc-2500/images"
source_dir = os.path.join(base_path, "minc-2500", "images")
# selected_classes = ['plastic', 'glass', 'metal', 'wood', 'paper']#,'water','polishedstone']

selected_classes = ['brick', 'carpet', 'ceramic', 'fabric', 'foliage', 'food', 'glass', 'hair', 'leather',
                    'metal', 'mirror', 'other', 'painted', 'paper', 'plastic', 'polishedstone', 'skin',
                    'sky', 'stone', 'tile', 'wallpaper', 'water', 'wood']

new_dir = "/kaggle/working/selected_images"

# Create new directory for selected classes
os.makedirs(new_dir, exist_ok=True)

# Copy only the selected classes to the new directory
for class_name in selected_classes:
    class_dir = os.path.join(source_dir, class_name)
    if os.path.isdir(class_dir):
        shutil.copytree(class_dir, os.path.join(new_dir, class_name))

# Now use splitfolders to split the selected classes


In [4]:
# 라이브러리 설치
!pip install split-folders

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import shutil
import timm
import splitfolders
import os

splitfolders.ratio(new_dir, output="/kaggle/working/Splitted", seed=1337, ratio=(0.85, 0.05, 0.10))

print("Selected classes have been split into train, validation, and test sets.")

train_path = '/kaggle/working/Splitted/train'
val_path = '/kaggle/working/Splitted/val'
test_path = '/kaggle/working/Splitted/test'

Copying files: 57500 files [00:32, 1762.27 files/s]

Selected classes have been split into train, validation, and test sets.


In [8]:
import torch
import torch.nn as nn
import timm
from torchvision.models import DenseNet121_Weights, densenet121
import os

class FrequencyFeatureExtractor(nn.Module):
    """
    2D FFT를 사용하여 이미지의 크기(Magnitude) 스펙트럼 특징을 추출하고 처리하는 모듈
    """
    def __init__(self, output_dim, image_size=224):
        super(FrequencyFeatureExtractor, self).__init__()
        self.image_size = image_size

        self.spectrum_processor = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        # 최종적으로 ViT 임베딩 차원(output_dim)에 맞추기 위한 투영 레이어
        self.projection = nn.Linear(64, output_dim)

    def forward(self, x):
        # 1. RGB 이미지를 그레이스케일로 변환
        x_gray = torch.mean(x, dim=1, keepdim=True)

        # 2. 2D FFT 적용 및 중심 이동
        f = torch.fft.fft2(x_gray, dim=(-2, -1))
        f_shifted = torch.fft.fftshift(f, dim=(-2, -1))

        # 3. 크기 스펙트럼 (Magnitude Spectrum) 계산
        magnitude_spectrum = torch.log(1 + torch.abs(f_shifted))

        # 4. 스펙트럼 처리 및 벡터화
        processed_features = self.spectrum_processor(magnitude_spectrum)
        processed_features = torch.flatten(processed_features, 1)

        # 5. ViT 임베딩 차원으로 투영
        frequency_features = self.projection(processed_features)

        return frequency_features


class FineTunedFusionClassifier(nn.Module):
    def __init__(self, num_classes=23, vit_model_name='vit_base_patch16_224', vit_weight_path=None):
        super(FineTunedFusionClassifier, self).__init__()

        # 1. ViT 모델 (timm)
        self.vit_model = timm.create_model(
            vit_model_name,
            pretrained=False,
            num_classes=0,
        )
        self.vit_embed_dim = self.vit_model.num_features # 768

        # ViT 가중치 로드
        if vit_weight_path and os.path.exists(vit_weight_path):
            print(f"INFO: Custom ViT weights loading from {vit_weight_path}...")
            custom_weights = torch.load(vit_weight_path)
            self.vit_model.load_state_dict(custom_weights, strict=False)

        # 2. DenseNet 특징 추출기
        self.cnn_model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
        self.cnn_features = self.cnn_model.features
        self.cnn_feature_dim = self.cnn_model.classifier.in_features # 1024

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # projection
        self.cnn_projection = nn.Sequential(
            nn.Linear(self.cnn_feature_dim, self.vit_embed_dim),
            nn.BatchNorm1d(self.vit_embed_dim),
            nn.GELU()
        )

        # 3. FFT feature extractor
        self.fft_extractor = FrequencyFeatureExtractor(output_dim=self.vit_embed_dim)

        total_fused_dim = 3 * self.vit_embed_dim

        self.classifier = nn.Sequential(
            nn.Linear(total_fused_dim, 1024),
            nn.BatchNorm1d(1024),  # BatchNorm
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),   # BatchNorm
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(512, num_classes)
        )

        self._set_trainable_layers()

    def _set_trainable_layers(self):
        # --- 기본적으로 모두 고정 ---
        for param in self.vit_model.parameters():
            param.requires_grad = False
        for param in self.cnn_features.parameters():
            param.requires_grad = False
        for name, module in self.fft_extractor.named_children():
             # spectrum_processor 등 내부 로직 고정
            for param in module.parameters():
                param.requires_grad = False

        # --- 일부 마지막 블록만 고정 해제 (Unfreeze) ---

        # 1. ViT의 마지막 1개 블록만 학습
        layers_to_train = 1
        print(f"INFO: ViT의 마지막 {layers_to_train}개 블록의 고정을 해제합니다.")
        for block in self.vit_model.blocks[-layers_to_train:]:
            for param in block.parameters():
                param.requires_grad = True

        # ViT의 Layer Norm(마지막 정규화)도 학습
        for param in self.vit_model.norm.parameters():
            param.requires_grad = True

        # 2. DenseNet의 마지막 DenseBlock만 학습
        # DenseNet121은 denseblock1 ~ denseblock4 구조입니다.
        print("INFO: DenseNet의 마지막 DenseBlock(4)의 고정을 해제합니다.")
        for param in self.cnn_features.denseblock4.parameters():
            param.requires_grad = True
        for param in self.cnn_features.norm5.parameters(): # 마지막 Norm
            param.requires_grad = True

        # --- projection, classifier 학습 ---
        for param in self.cnn_projection.parameters():
            param.requires_grad = True

        # FFT Extractor 내부의 projection 레이어가 있다면 학습
        if hasattr(self.fft_extractor, 'projection'):
            for param in self.fft_extractor.projection.parameters():
                param.requires_grad = True

        for param in self.classifier.parameters():
            param.requires_grad = True

    def forward(self, x):
        # 1. ViT (CLS Token)
        vit_features = self.vit_model.forward_features(x)
        cls_token = vit_features[:, 0, :] # (B, 768)

        # 2. DenseNet
        cnn_features_map = self.cnn_features(x)
        cnn_features_vec = torch.flatten(self.global_avg_pool(cnn_features_map), 1)
        cnn_frequency_features = self.cnn_projection(cnn_features_vec) # (B, 768)

        # 3. FFT
        fft_frequency_features = self.fft_extractor(x) # (B, 768)

        # 4. Fusion
        fused_features = torch.cat((cls_token, cnn_frequency_features, fft_frequency_features), dim=1)

        # 5. Classifier
        logits = self.classifier(fused_features)

        return logits

In [14]:
import torchvision.transforms.functional as TF

class SetFixedContrast(object):
    """
    이미지의 대비(Contrast)를 특정 상수 값으로 고정하여 설정합니다.
    """
    def __init__(self, factor=0.1):
        self.factor = factor

    def __call__(self, img):
        # PIL Image를 입력으로 받아 Contrast를 factor로 조정
        return TF.adjust_contrast(img, self.factor)

In [34]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
import os
from PIL import Image
import numpy as np
# OpenCV가 필요하므로 import 합니다.
import cv2

# ==========================================
# 1. 설정 (Configuration)
# ==========================================
# splitfolders로 생성된 TEST 경로
TEST_DIR = '/kaggle/working/Splitted/test'

# 학습된 모델 경로 (Fine-tuning 결과 파일)
MODEL_PATH = '/content/drive/MyDrive/ComputerVision/best_material_classifier_epoch_6.pth'

BATCH_SIZE = 32
NUM_CLASSES = 23
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ==========================================
# 2. 실험용 커스텀 Transform 정의
# ==========================================

class FrequencyFeatureExtractor(nn.Module):
    """
    2D FFT를 사용하여 이미지의 크기(Magnitude) 스펙트럼 특징을 추출하고 처리하는 모듈
    """
    def __init__(self, output_dim, image_size=224):
        super(FrequencyFeatureExtractor, self).__init__()
        self.image_size = image_size

        self.spectrum_processor = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        # 최종적으로 ViT 임베딩 차원(output_dim)에 맞추기 위한 투영 레이어
        self.projection = nn.Linear(64, output_dim)

    def forward(self, x):
        # 1. RGB 이미지를 그레이스케일로 변환
        x_gray = torch.mean(x, dim=1, keepdim=True)

        # 2. 2D FFT 적용 및 중심 이동
        f = torch.fft.fft2(x_gray, dim=(-2, -1))
        f_shifted = torch.fft.fftshift(f, dim=(-2, -1))

        # 3. 크기 스펙트럼 (Magnitude Spectrum) 계산
        magnitude_spectrum = torch.log(1 + torch.abs(f_shifted))

        # 4. 스펙트럼 처리 및 벡터화
        processed_features = self.spectrum_processor(magnitude_spectrum)
        processed_features = torch.flatten(processed_features, 1)

        # 5. ViT 임베딩 차원으로 투영
        frequency_features = self.projection(processed_features)

        return frequency_features

# 2-2. Fine-Tuned Fusion Classifier (학습 코드에서 가져옴)
class FineTunedFusionClassifier(nn.Module):
    def __init__(self, num_classes=23, vit_model_name='vit_base_patch16_224', vit_weight_path=None):
        super(FineTunedFusionClassifier, self).__init__()

        # 1. ViT 모델 (timm)
        self.vit_model = timm.create_model(
            vit_model_name,
            pretrained=False, # 학습 시와 동일하게
            num_classes=0,
        )
        self.vit_embed_dim = self.vit_model.num_features # 768

        # ViT 가중치 로드 부분은 테스트에서는 건너뜁니다.

        # 2. DenseNet 특징 추출기
        self.cnn_model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
        self.cnn_features = self.cnn_model.features
        self.cnn_feature_dim = self.cnn_model.classifier.in_features # 1024

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # projection
        self.cnn_projection = nn.Sequential(
            nn.Linear(self.cnn_feature_dim, self.vit_embed_dim),
            nn.BatchNorm1d(self.vit_embed_dim),
            nn.GELU()
        )

        # 3. FFT feature extractor (위에 정의한 클래스 사용)
        self.fft_extractor = FrequencyFeatureExtractor(output_dim=self.vit_embed_dim)

        total_fused_dim = 3 * self.vit_embed_dim

        self.classifier = nn.Sequential(
            nn.Linear(total_fused_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(512, num_classes)
        )

        # _set_trainable_layers 함수는 가중치 로드에 영향을 주지 않으므로 정의만 유지
        self._set_trainable_layers()

    def _set_trainable_layers(self):
        # 학습 코드의 내용을 그대로 복사합니다.
        for param in self.vit_model.parameters():
            param.requires_grad = False
        for param in self.cnn_features.parameters():
            param.requires_grad = False
        for name, module in self.fft_extractor.named_children():
            for param in module.parameters():
                param.requires_grad = False

        layers_to_train = 1
        for block in self.vit_model.blocks[-layers_to_train:]:
            for param in block.parameters():
                param.requires_grad = True

        for param in self.vit_model.norm.parameters():
            param.requires_grad = True

        for param in self.cnn_features.denseblock4.parameters():
            param.requires_grad = True
        for param in self.cnn_features.norm5.parameters():
            param.requires_grad = True

        for param in self.cnn_projection.parameters():
            param.requires_grad = True

        if hasattr(self.fft_extractor, 'projection'):
            for param in self.fft_extractor.projection.parameters():
                param.requires_grad = True

        for param in self.classifier.parameters():
            param.requires_grad = True

    def forward(self, x):
        # 1. ViT (CLS Token)
        vit_features = self.vit_model.forward_features(x)
        cls_token = vit_features[:, 0, :]

        # 2. DenseNet
        cnn_features_map = self.cnn_features(x)
        cnn_features_vec = torch.flatten(self.global_avg_pool(cnn_features_map), 1)
        cnn_frequency_features = self.cnn_projection(cnn_features_vec)

        # 3. FFT
        fft_frequency_features = self.fft_extractor(x)

        # 4. Fusion
        fused_features = torch.cat((cls_token, cnn_frequency_features, fft_frequency_features), dim=1)

        # 5. Classifier
        logits = self.classifier(fused_features)

        return logits


# ==========================================
# 3. 실험용 커스텀 Transform 정의 (변경 없음)
# ==========================================

class AddGaussianNoise(object):
    """이미지 텐서에 노이즈를 추가하는 클래스"""
    def __init__(self, mean=0., std=0.1):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

# Shape 제거: 64x64 패치 랜덤 셔플
class RemoveShapeByPatchShuffle(object):
    """
    이미지를 64x64 패치로 나누고 랜덤하게 섞어 형태 정보를 제거합니다.
    입력: PIL Image (256x256 크기가 예상됨)
    출력: PIL Image
    """
    def __init__(self, patch_size=64):
        self.patch_size = patch_size

    def __call__(self, img):
        # PIL Image를 NumPy 배열로 변환
        img_np = np.array(img)
        H, W, C = img_np.shape
        P = self.patch_size

        # 256x256으로 리사이즈했다면 H와 W는 256이어야 함
        if H % P != 0 or W % P != 0:
            # 리사이즈가 제대로 안 됐다면, 로직 오류 방지를 위해 경고 대신 에러 발생 또는 64의 배수로 강제 조정
            print(f"CRITICAL WARNING: Image size {H}x{W} is NOT divisible by patch size {P}. Skipping shuffle.")
            return img # 셔플 없이 원본 반환 (실험 조건 위반)

        # 이미지 패치로 나누기, 셔플, 재구성 로직은 동일
        # ... (이하 패치 셔플 로직)
        patches = img_np.reshape(H // P, P, W // P, P, C)
        patches = patches.swapaxes(1, 2)
        patches = patches.reshape(-1, P, P, C)

        indices = np.arange(len(patches))
        np.random.shuffle(indices)
        shuffled_patches = patches[indices]

        num_H_patches = H // P
        num_W_patches = W // P

        shuffled_patches = shuffled_patches.reshape(num_H_patches, num_W_patches, P, P, C)
        shuffled_patches = shuffled_patches.swapaxes(1, 2)
        reconstructed_img_np = shuffled_patches.reshape(H, W, C)

        reconstructed_img_np = reconstructed_img_np.astype(np.uint8)

        # NumPy 배열을 다시 PIL Image로 변환
        return Image.fromarray(reconstructed_img_np)



# Texture 제거 후 Edge 복원 (OpenCV 사용)
class TextureRemovalEdgeRestoration(object):
    def __call__(self, img):
        img_bgr = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

        # ① Texture 제거: Gaussian Blur (9x9 커널)
        blur = cv2.GaussianBlur(img_bgr, (9, 9), 0)

        # ② Edge 복원: Canny Edge Detection
        blur_gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(blur_gray, 100, 200)

        # 다시 3채널 BGR로 변환
        edges_bgr = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

        # ③ 최종 합성 단계: Weighted Addition
        result_bgr = cv2.addWeighted(blur, 0.8, edges_bgr, 0.2, 0)

        return Image.fromarray(cv2.cvtColor(result_bgr, cv2.COLOR_BGR2RGB))

# Color 제거 (Grayscale 변환 후 3채널 복제)
class RemoveColorToGrayscale(object):
    def __call__(self, img):
        img_gray = img.convert('L')
        img_np = np.array(img_gray)
        img_3channel = np.stack((img_np,)*3, axis=-1)
        return Image.fromarray(img_3channel)


def get_transforms(mode='clean'):
    """ mode에 따라 다른 전처리를 반환 """

    final_transforms = [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]

    if mode == 'clean':
        return transforms.Compose(final_transforms)

    elif mode == 'illumination':
        # 조명 변화: Contrast를 0.1로 고정
        return transforms.Compose([
            # Contrast를 0.1로 고정하여 적용
            transforms.Resize((224, 224)), # 먼저 모델 입력 크기로 리사이즈
            SetFixedContrast(factor=0.1), # 대비 0.1로 고정 (매우 낮은 대비)
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    elif mode == 'shape_removed':
        # 1. 256x256으로 리사이징 (64로 나누어 떨어지게 하기 위함)
        # 2. 64x64 패치 셔플
        # 3. 최종 크기(224x224)로 리사이징 및 텐서 변환/정규화
        return transforms.Compose([
            transforms.Resize((256, 256)),
            RemoveShapeByPatchShuffle(patch_size=64), # 64x64 패치 사용
            *final_transforms
        ])

    elif mode == 'texture_edge':
        return transforms.Compose([
            TextureRemovalEdgeRestoration(),
            *final_transforms
        ])

    elif mode == 'color_removed':
        return transforms.Compose([
            RemoveColorToGrayscale(),
            *final_transforms
        ])



    else:
        raise ValueError(f"Unknown mode: {mode}")

# ==========================================
# 4. 평가 함수 (변경 없음)
# ==========================================
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

# ==========================================
# 5. 메인 실행 코드 (Fusion Classifier 로드)
# ==========================================
if __name__ == '__main__':
    print(f"Using device: {device}")

    # 1. 모델 구조 생성: FineTunedFusionClassifier를 로드합니다.
    model = FineTunedFusionClassifier(num_classes=NUM_CLASSES)

    # 2. 학습된 가중치 로드
    if os.path.exists(MODEL_PATH):
        print(f"Loading Fine-tuned FusionClassifier weights from {MODEL_PATH}...")

        state_dict = torch.load(MODEL_PATH, map_location=device)

        # 가중치 파일이 'model' 키 아래에 저장된 경우 처리
        if 'model' in state_dict:
            state_dict = state_dict['model']

        # FusionClassifier 구조에 전체 가중치를 로드합니다. (오류 해결)
        try:
            model.load_state_dict(state_dict, strict=True)
            print("INFO: Model weights loaded successfully.")
        except RuntimeError as e:
            print(f"CRITICAL ERROR: Failed to load state_dict for FusionClassifier.")
            print(f"Error details: {e}")
            exit()

    else:
        print(f"Error: {MODEL_PATH} 파일을 찾을 수 없습니다! 학습 먼저 완료해주세요.")
        exit()

    model.to(device)

    # 3. 실험 모드별 테스트 실행
    modes = ['clean', 'shape_removed', 'texture_edge', 'color_removed', 'illumination']
    #modes = ['clean','illumination']

    results = {}

    print("\n" + "="*50)
    print("       STARTING MATERIAL CLASSIFICATION ROBUSTNESS TEST      ")
    print("="*50)

    for mode in modes:
        print(f"\nTesting Mode: [{mode.upper()}]")

        transform = get_transforms(mode)

        try:
            test_dataset = datasets.ImageFolder(root=TEST_DIR, transform=transform)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

            if len(test_dataset) == 0:
                print("Error: Test 폴더가 비어있습니다. 데이터를 확인해주세요.")
                continue

            acc = evaluate(model, test_loader)
            results[mode] = acc
            print(f"--> Accuracy: {acc:.2f}%")

        except FileNotFoundError:
            print(f"Error: {TEST_DIR} 경로를 찾을 수 없습니다. 경로를 확인해주세요.")
            break

    # 4. 최종 결과 리포트
    print("\n" + "="*50)
    print("           FINAL RESULTS REPORT        ")
    print("="*50)

    base_acc = results.get('clean', 0.0)
    print(f"Original Accuracy (Clean): {base_acc:.2f}%")

    for mode in modes:
        if mode == 'clean': continue
        if mode in results:
            acc = results[mode]
            drop = base_acc - acc
            print(f"- {mode.upper()} Accuracy: {acc:.2f}%  (Drop: -{drop:.2f}%)")

    print("="*50)

Using device: cuda
Loading Fine-tuned FusionClassifier weights from /content/drive/MyDrive/ComputerVision/best_material_classifier_epoch_6.pth...
INFO: Model weights loaded successfully.

       STARTING MATERIAL CLASSIFICATION ROBUSTNESS TEST      

Testing Mode: [CLEAN]
--> Accuracy: 88.56%

Testing Mode: [SHAPE_REMOVED]
--> Accuracy: 46.96%

Testing Mode: [TEXTURE_EDGE]
--> Accuracy: 84.33%

Testing Mode: [COLOR_REMOVED]
--> Accuracy: 82.28%

Testing Mode: [ILLUMINATION]
--> Accuracy: 62.03%

           FINAL RESULTS REPORT        
Original Accuracy (Clean): 88.56%
- SHAPE_REMOVED Accuracy: 46.96%  (Drop: -41.60%)
- TEXTURE_EDGE Accuracy: 84.33%  (Drop: -4.23%)
- COLOR_REMOVED Accuracy: 82.28%  (Drop: -6.28%)
- ILLUMINATION Accuracy: 62.03%  (Drop: -26.52%)


# 아래는 샘플 이미지 뽑기용

In [33]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
import os
from PIL import Image
import numpy as np
import cv2
import torchvision.transforms.functional as TF # Contrast 고정에 필요

# ==========================================
# 1. 설정 (Configuration)
# ==========================================
# splitfolders로 생성된 TEST 경로
TEST_DIR = '/kaggle/working/Splitted/test'

SPECIFIC_SAMPLE_PATH = '/content/drive/MyDrive/ComputerVision/brick_000006.jpg'

# 학습된 모델 경로 (Fine-tuning 결과 파일)
MODEL_PATH = '/content/drive/MyDrive/ComputerVision/best_material_classifier_epoch_6.pth'

BATCH_SIZE = 32
NUM_CLASSES = 23
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 이미지 저장을 위한 디렉토리 생성
OUTPUT_DIR = './transformed_samples'
os.makedirs(OUTPUT_DIR, exist_ok=True)


# ==========================================
# 2. 모델 및 커스텀 Transform 정의 (일부 생략 및 추가)
# ==========================================

# ... (FrequencyFeatureExtractor 및 FineTunedFusionClassifier 클래스는 그대로 유지) ...
from torchvision.models import DenseNet121_Weights, densenet121
# 이 부분은 사용자님의 코드 상단에 포함되어야 합니다.

# ==========================================
# 2-A. ILLUMINATION 모드용 커스텀 Transform 추가
# ==========================================
class SetFixedContrast(object):
    """
    이미지의 대비(Contrast)를 특정 상수 값으로 고정하여 설정합니다.
    """
    def __init__(self, factor=0.1):
        self.factor = factor

    def __call__(self, img):
        # PIL Image를 입력으로 받아 Contrast를 factor로 조정
        return TF.adjust_contrast(img, self.factor)


# ==========================================
# 3. 실험용 커스텀 Transform 정의 (get_transforms 함수만 수정)
# ==========================================

# ... (AddGaussianNoise, RemoveShapeByPatchShuffle 등 클래스 정의는 그대로 유지) ...


def get_transforms(mode='clean'):
  """ mode에 따라 다른 전처리를 반환 """

  final_transforms = [transforms.Resize((224, 224)),transforms.ToTensor(),
                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
  if mode == 'clean':
    return transforms.Compose(final_transforms)

  elif mode == 'illumination':
    # 조명 변화: Contrast를 0.1로 고정
    return transforms.Compose([
        # Contrast 0.1 고정 시 PIL Image를 처리해야 하므로 Resize를 먼저 적용합니다.
        transforms.Resize((224, 224)),
        SetFixedContrast(factor=0.1), # 대비 0.1로 고정 (매우 낮은 대비)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

  elif mode == 'shape_removed':
    # 1. 256x256으로 리사이징 (64로 나누어 떨어지게 하기 위함)
    # 2. 64x64 패치 셔플
    # 3. 최종 크기(224x224)로 리사이징 및 텐서 변환/정규화
    return transforms.Compose([
        transforms.Resize((256, 256)),
        RemoveShapeByPatchShuffle(patch_size=64), # 64x64 패치 사용
        *final_transforms])

  elif mode == 'texture_edge':
    return transforms.Compose([
        TextureRemovalEdgeRestoration(),
        *final_transforms])

  elif mode == 'color_removed':
    return transforms.Compose([
        RemoveColorToGrayscale(),
        *final_transforms
        ])

  else:
    raise ValueError(f"Unknown mode: {mode}")


# ==========================================
# 4. 이미지 샘플 저장 함수 (새로 추가)
# ==========================================

def save_sample_image(mode, transform, specific_file_path, output_dir='./transformed_samples'):
    """
    지정된 Google Drive 파일 경로의 이미지를 찾아 변환을 적용하고 저장합니다.
    """

    # 1. Google Drive 경로에서 이미지 로드
    try:
        # 경로에서 파일 이름과 기본 이름 추출
        file_name = os.path.basename(specific_file_path)
        base_name = os.path.splitext(file_name)[0]

        # PIL Image를 로드
        sample_img_pil = Image.open(specific_file_path).convert('RGB')

    except Exception as e:
        print(f"CRITICAL ERROR: Failed to load sample image from {specific_file_path}.")
        print(f"Error details: {e}")
        return

    # 2. 변환 적용 (ToTensor()와 Normalize()를 제외한 PIL 단계의 Transform)
    try:
        # NOTE: 이 부분은 get_transforms 함수와 로직이 동일해야 합니다.
        if mode == 'clean':
            display_transforms = transforms.Resize((224, 224))

        elif mode == 'illumination':
            display_transforms = transforms.Compose([
                transforms.Resize((224, 224)),
                SetFixedContrast(factor=0.1),
            ])

        elif mode == 'shape_removed':
            display_transforms = transforms.Compose([
                transforms.Resize((256, 256)),
                RemoveShapeByPatchShuffle(patch_size=4),
                transforms.Resize((224, 224)),
            ])

        elif mode == 'texture_edge':
            display_transforms = transforms.Compose([
                TextureRemovalEdgeRestoration(),
                transforms.Resize((224, 224)),
            ])

        elif mode == 'color_removed':
            display_transforms = transforms.Compose([
                RemoveColorToGrayscale(),
                transforms.Resize((224, 224)),
            ])

        else:
            print(f"WARNING: Unknown mode {mode} for saving sample.")
            return

        transformed_img_pil = display_transforms(sample_img_pil)

        # 3. 이미지 저장
        os.makedirs(output_dir, exist_ok=True)
        save_path = os.path.join(output_dir, f"{base_name}_{mode.upper()}.png")
        transformed_img_pil.save(save_path)

        print(f"--> Saved sample image for [{mode.upper()}] to: {save_path}")

    except Exception as e:
        print(f"Error applying transform or saving sample for {mode.upper()}: {e}")

    return

# ==========================================
# 5. 메인 실행 코드 (Fusion Classifier 로드 및 샘플 저장)
# ==========================================
if __name__ == '__main__':
    # ... (모델 로드 코드는 그대로 유지) ...
    # 3. 실험 모드별 테스트 실행
    #modes = ['clean', 'shape_removed', 'texture_edge', 'color_removed', 'illumination']
    modes = ['shape_removed']
    results = {}

    print("\n" + "="*50)
    print("       STARTING MATERIAL CLASSIFICATION ROBUSTNESS TEST      ")
    print("="*50)
    print(f"Sample images will be saved in: {OUTPUT_DIR}")

    for mode in modes:
        print(f"\nTesting Mode: [{mode.upper()}]")

        transform = get_transforms(mode)

        # 🌟 모드별 변환된 샘플 이미지 저장 🌟
        #save_sample_image(mode, transform)
        save_sample_image(mode, transform, SPECIFIC_SAMPLE_PATH, OUTPUT_DIR)

        try:
            test_dataset = datasets.ImageFolder(root=TEST_DIR, transform=transform)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

            if len(test_dataset) == 0:
                print("Error: Test 폴더가 비어있습니다. 데이터를 확인해주세요.")
                continue

            acc = evaluate(model, test_loader)
            results[mode] = acc
            print(f"--> Accuracy: {acc:.2f}%")

        except FileNotFoundError:
            print(f"Error: {TEST_DIR} 경로를 찾을 수 없습니다. 경로를 확인해주세요.")
            break

    # 4. 최종 결과 리포트... (이하 동일)


       STARTING MATERIAL CLASSIFICATION ROBUSTNESS TEST      
Sample images will be saved in: ./transformed_samples

Testing Mode: [SHAPE_REMOVED]
--> Saved sample image for [SHAPE_REMOVED] to: ./transformed_samples/brick_000006_SHAPE_REMOVED.png
--> Accuracy: 46.05%
